# This notebook was used to convert the excel table to a Latex table

In [ ]:
from astropy.io import fits
import numpy as np
from scipy.odr import *
from astropy.coordinates import SkyCoord
from astroquery.vo_conesearch import ConeSearch
from astroquery.gaia import Gaia
from scipy.ndimage import interpolation
from astropy import units as u
from astropy import constants as c
import astropy.coordinates as coord
from astropy import wcs
import aplpy
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy.coordinates import ICRS,FK4
import statistics    
import math
import re
import os
import pandas as pd

# Create Source overview table

In [19]:
# Load in data
tbl = pd.read_csv('preprocessed_data_avg.csv')
display(tbl.head())

SO = pd.read_csv('FSRED Mags - SourceOverviewOverleaf.csv')
display(SO.head())

,Unnamed: 0,Source,Filter,Obs_number,Detected Name,Date obs,RA,DEC,Error_circle (arcsec),Position_source,...,log_L_NIR_errup_nodist,log_L_NIR_errlow_nodist,log_L_NIR_errup_noflux,log_L_NIR_errlow_noflux,log_L_X_errup_nodist,log_L_X_errlow_nodist,log_L_X_errup_noflux,log_L_X_errlow_noflux,log_L_NIR_upplim,log_L_X_upplim
0,0,RX_J1735.3-3540,J,3081,S,2014-05-07,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.003808,0.003808,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN
1,2,RX_J1735.3-3540,H,3063,S,2014-05-07,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.002848,0.002848,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN
2,4,RX_J1735.3-3540,Ks,3045,S,2014-05-07,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.003019,0.003019,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN
3,6,UW_Crb,Ks,1081,S,2018-06-19,16 05 45.8746697208,25 51 45.246497976,0.10,"Gaia Collaboration 2020, Gaia",...,0.051416,0.051416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,UW_Crb,H,1086,S,2018-06-19,16 05 45.8746697208,25 51 45.246497976,0.10,"Gaia Collaboration 2020, Gaia",...,0.038230,0.038230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Source,RA,DEC,poserror,poslit,NS/BH,Transient/persistent,Hard/soft state,State_source,nH,Distance,D_errup,D_errlow,Distance_source,Note
0,1E_1743.1-2843,NaN,NaN,NaN,NaN,NS,persistent,hard,NaN,NaN,NaN,NaN,NaN,NaN,Multiple counterparts
1,2E_17467-3225,NaN,NaN,NaN,NaN,NS,persistent,hard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2S_0918-549,NaN,NaN,NaN,NaN,NS,persistent,hard,Muñoz-Daria+2014,NaN,NaN,NaN,NaN,NaN,NaN
3,4U_1323-619,NaN,NaN,NaN,NaN,NS,persistent,hard,Muñoz-Daria+2014,NaN,NaN,NaN,NaN,NaN,Source blended
4,4U_1626-67,NaN,NaN,NaN,NaN,NS,persistent,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
for index, row in SO.iterrows():
    source = row['Source']
    
    if source in tbl['Source'].tolist():
        RA = tbl[tbl['Source']==source]['RA'].tolist()[0]
        SO.loc[index, 'RA'] = RA
        
        Dec = tbl[tbl['Source']==source]['DEC'].tolist()[0]
        SO.loc[index, 'DEC'] = Dec

        Error = tbl[tbl['Source']==source]['Error_circle (arcsec)'].tolist()[0]
        SO.loc[index, 'poserror'] = Error
        
        poslit = tbl[tbl['Source']==source]['Position_source'].tolist()[0]
        SO.loc[index, 'poslit'] = poslit
        
        
        nh = tbl[tbl['Source']==source]['nH'].tolist()[0]
        SO.loc[index, 'nH'] = nh

        Distance = tbl[tbl['Source']==source]['Distance (kpc)'].tolist()[0]
        SO.loc[index, 'Distance'] = Distance
   
        dup = tbl[tbl['Source']==source]['D_errup'].tolist()[0]
        SO.loc[index, 'D_errup'] = dup
        
        dlow = tbl[tbl['Source']==source]['D_errlow'].tolist()[0]
        SO.loc[index, 'D_errlow'] = dlow
           
        dlit = tbl[tbl['Source']==source]['Distance_source'].tolist()[0]
        SO.loc[index, 'Distance_source'] = dlit

In [21]:
display(SO.head())

,Source,RA,DEC,poserror,poslit,NS/BH,Transient/persistent,Hard/soft state,State_source,nH,Distance,D_errup,D_errlow,Distance_source,Note
0,1E_1743.1-2843,NaN,NaN,NaN,NaN,NS,persistent,hard,NaN,NaN,NaN,NaN,NaN,NaN,Multiple counterparts
1,2E_17467-3225,17 50 3.95,-32 25 50.1,0.6,"Wilson+2003, Chandra",NS,persistent,hard,NaN,5.010000e+21,8.0,NaN,NaN,NaN,NaN
2,2S_0918-549,09 20 26.4818170056,-55 12 24.512806404,0.1,"Gaia Collaboration 2020, Gaia",NS,persistent,hard,Muñoz-Daria+2014,5.290000e+21,5.4,NaN,NaN,in 't Zand+2005,NaN
3,4U_1323-619,13 26 37.03,-62 08 08.88,0.6,"Gambino+2015, Chandra",NS,persistent,hard,Muñoz-Daria+2014,1.170000e+22,4.2,0.8,0.8,Gambino+2015,Source blended
4,4U_1626-67,16 32 16.7,-67 27 40.3,1.0,"Watson+2008, XMMN",NS,persistent,unknown,NaN,9.600000e+20,9.0,4.0,4.0,Chakrabarty 1998,NaN


In [22]:
SO.to_csv('FSRED Mags - SourceOverviewOverleaf_filled.csv')


# Create observation overview table


In [35]:
# tblpy = pd.read_csv('FSRED Mags - Final_python.csv')
tblpre = pd.read_csv('preprocessed_data2.csv')
tblpre.head()

,Unnamed: 0,Source,Filter,Obs_number,Detected Name,Date obs,RA,DEC,Error_circle (arcsec),Position_source,...,log_L_NIR_errup_nodist,log_L_NIR_errlow_nodist,log_L_NIR_errup_noflux,log_L_NIR_errlow_noflux,log_L_X_errup_nodist,log_L_X_errlow_nodist,log_L_X_errup_noflux,log_L_X_errlow_noflux,log_L_NIR_upplim,log_L_X_upplim
0,0,RX_J1735.3-3540,J,3081,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.002626,0.002626,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN
1,1,RX_J1735.3-3540,J,3090,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.002762,0.002762,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN
2,2,RX_J1735.3-3540,H,3063,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.002013,0.002013,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN
3,3,RX_J1735.3-3540,H,3072,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.002004,0.002004,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN
4,4,RX_J1735.3-3540,Ks,3045,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,0.002104,0.002104,NaN,NaN,0.108574,0.108574,NaN,NaN,NaN,NaN


In [ ]:
def round_with_error(number, error):
    decimal_places = -int(math.floor(math.log10(error)))
    rounded_number = round(number, decimal_places)
    return rounded_number

In [137]:

magraw_values = []
magrawerr = []
magcorr_values = []
magcorrerr = []
Lnir_values = []
Lnir_errup = []
Lnir_errlo = []
Lx_values = []
Lx_errup = []
Lx_errlo = []
list_exporg = []
list_ncoadd = []
obs_list = []
av_list = []
zpo_list = []

for index, row in tblpre.iterrows():

    # Parameters
    source = str(row['Source'])
    obs =  str(row['Obs_number'])
    if len(obs) ==3:
        obs = str(0) + obs
    obs_list.append(obs)
    date = str(row['Date obs'])
    band = str(row['Filter'])

    # Create filepaths
    FSRED_dir = "E:/Scriptie/"
    source_dir = FSRED_dir + "FSRED/u018918_data/" + source + "/" 
    file_name_start = date + '_' + band + '_' #+ source + '_'+ obs + '_' 
    # List all files in the directory
    files_in_directory = os.listdir(source_dir)
    # Filter files based on the starting name
    matching_files = [file for file in files_in_directory if file.startswith(file_name_start) and file.endswith('.fits')]
    for f in matching_files: 
        if obs in f:
            file = f

    # Obtain exposure info from header
    hdul = fits.open(source_dir + f)
    hdr = hdul[0].header
    exporg = hdr['EXPORG']
    ncoadd = hdr['NCOADD']
    list_exporg.append(exporg)
    list_ncoadd.append(ncoadd)
    
    # Av and ZPO
    av = row['An_extinction']
    av_list.append("{:.2f}".format(av))
    zpo = row['zp_offset']
    zpo_list.append("{:.3f}".format(zpo))
    
    # Magnitude
    if not np.isnan(row['Upper_limit']):
        print('upplim source:' + source)
        magraw_values.append('>' + "{:.3f}".format(row['Upper_limit'] + row['zp_offset']))
        magcorr_values.append('>' + "{:.3f}".format(row['Upper_limit'] - row['An_extinction'])) 
        magrawerr.append('DUMMY')
        magcorrerr.append('DUMMY')
    else:
        magraw_values.append("{:.3f}".format(row['Mag_raw']))
        magrawerr.append("{:.3f}".format(row['Mag_err']))
        magcorr_values.append("{:.3f}".format(row['Mag_corr']))
        magcorrerr.append("{:.3f}".format(row['Mag_err_corr']))
        
    # luminosity
    if not np.isnan(row['Upper_limit']): 
        Lnir =  f"{row['L_NIR_upplim']:.2e}"
        Lnir_values.append('<' + str(Lnir))
        Lnir_errup.append('DUMMY')
        Lnir_errlo.append('DUMMY')
    else:
        Lnir =  f"{row['L_NIR']:.2e}"
        Lnir_values.append(str(Lnir))
        Lnir_errup.append(row['L_NIR_errup'])
        Lnir_errlo.append(row['L_NIR_errlow'])        
        
    # X-ray
    if not np.isnan(row['Xray_upplim']):
        print('no xray flux for' + str(source))
        Lx_values.append('<' + str(row['Xray_upplim']))
        Lx_errup.append('DUMMY')
        Lx_errlo.append('DUMMY')
    elif not np.isnan(row['Xray_Flux (erg/s/cm2)']):
        Lx_values.append(row['Xray_Flux (erg/s/cm2)'])
        Lx_errup.append(row['Xray_errup'])
        Lx_errlo.append(row['Xray_errlow'])
    else: 
        Lx_values.append('DUMMY')
        Lx_errup.append('DUMMY')
        Lx_errlo.append('DUMMY')               

upplim source:XTE_J1719-291
no xray flux forXTE_J1719-291
upplim source:XTE_J1719-291
no xray flux forXTE_J1719-291
upplim source:EXO_0748-676
upplim source:EXO_0748-676
upplim source:EXO_0748-676
upplim source:IGR_J17494-3030
no xray flux forIGR_J17494-3030
no xray flux forMAXI_J1305-704
no xray flux forMAXI_J1305-704
no xray flux forMAXI_J1305-704
no xray flux forMAXI_J1305-704
no xray flux forMAXI_J1305-704
upplim source:2E_17467-3225
upplim source:RXS_J180408.9-342058
upplim source:RXS_J171824.2-402934
upplim source:RXS_J171824.2-402934
upplim source:RXS_J171824.2-402934
upplim source:Swift_J1922.7-1716
upplim source:Swift_J1922.7-1716
upplim source:Swift_J1922.7-1716
upplim source:SAX_J1712.6-3739
upplim source:MAXI_J1421-613
no xray flux forMAXI_J1421-613
upplim source:MAXI_J1421-613
no xray flux forMAXI_J1421-613
upplim source:XTE_J1728-295
no xray flux forXTE_J1728-295
upplim source:XTE_J1901+014


In [138]:
tblpre['finalmagraw'] = magraw_values
tblpre['finalmagrawerr'] = magrawerr 
tblpre['finalmagcor'] = magcorr_values 
tblpre['finalmagcorerr'] = magcorrerr 
tblpre['finalLnir'] = Lnir_values 
tblpre['finalLnirerrup'] = Lnir_errup 
tblpre['finalLnirerrlo'] = Lnir_errlo 
tblpre['finalLX'] = Lx_values 
tblpre['finalLXerrup'] = Lx_errup 
tblpre['finalLXerrlo'] = Lx_errlo 


tblpre['obsfinal'] = obs_list
tblpre['zpofinal'] = zpo_list
tblpre['avfinal'] = av_list

tblpre['ncoadd'] = list_ncoadd
tblpre['exporg'] = list_exporg

display(tblpre.head())

,Unnamed: 0,Source,Filter,Obs_number,Detected Name,Date obs,RA,DEC,Error_circle (arcsec),Position_source,...,finalLnirerrup,finalLnirerrlo,finalLX,finalLXerrup,finalLXerrlo,ncoadd,exporg,obsfinal,zpofinal,avfinal
0,0,RX_J1735.3-3540,J,3081,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,NaN,NaN,0.0,0.0,0.0,1.0,2.911,3081,-0.233,0.91
1,1,RX_J1735.3-3540,J,3090,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,NaN,NaN,0.0,0.0,0.0,1.0,2.911,3090,-0.092,0.91
2,2,RX_J1735.3-3540,H,3063,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,NaN,NaN,0.0,0.0,0.0,1.0,5.822,3063,-2.642,0.56
3,3,RX_J1735.3-3540,H,3072,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,NaN,NaN,0.0,0.0,0.0,1.0,5.822,3072,-1.101,0.56
4,4,RX_J1735.3-3540,Ks,3045,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",...,NaN,NaN,0.0,0.0,0.0,1.0,2.911,3045,-1.017,0.36


In [139]:
paper_table = tblpre.copy()
columns = ['Source', 'Filter', 'obsfinal', 'Date obs', 'exporg', 'ncoadd', 'finalmagraw', 'finalmagrawerr', 'avfinal', 'zpofinal', 'finalmagcor', 'finalmagcorerr', 'finalLnir', 'finalLnirerrup', 'finalLnirerrlo', 'finalLX', 'finalLXerrup', 'finalLXerrlo', 'Xray_Flux_source']

In [140]:
paper_table = paper_table[columns]
display(paper_table.head())

,Source,Filter,obsfinal,Date obs,exporg,ncoadd,finalmagraw,finalmagrawerr,avfinal,zpofinal,finalmagcor,finalmagcorerr,finalLnir,finalLnirerrup,finalLnirerrlo,finalLX,finalLXerrup,finalLXerrlo,Xray_Flux_source
0,RX_J1735.3-3540,J,3081,2014-05-08,2.911,1.0,15.006,0.007,0.91,-0.233,14.331,0.017,3.20e+34,NaN,NaN,0.0,0.0,0.0,Thesis Bilal 2018
1,RX_J1735.3-3540,J,3090,2014-05-08,2.911,1.0,15.193,0.007,0.91,-0.092,14.378,0.017,3.06e+34,NaN,NaN,0.0,0.0,0.0,Thesis Bilal 2018
2,RX_J1735.3-3540,H,3063,2014-05-08,5.822,1.0,11.521,0.005,0.56,-2.642,13.600,0.012,4.59e+34,NaN,NaN,0.0,0.0,0.0,Thesis Bilal 2018
3,RX_J1735.3-3540,H,3072,2014-05-08,5.822,1.0,12.899,0.005,0.56,-1.101,13.437,0.012,5.33e+34,NaN,NaN,0.0,0.0,0.0,Thesis Bilal 2018
4,RX_J1735.3-3540,Ks,3045,2014-05-08,2.911,1.0,12.577,0.005,0.36,-1.017,13.234,0.010,4.17e+34,NaN,NaN,0.0,0.0,0.0,Thesis Bilal 2018


In [141]:
paper_table.to_excel('paper_table.xlsx')


In [143]:
# TODO determine luminosities based on flux and distance
def luminosity(flux, d):
    d_cm = 3.08567758128e21 * float(d) 
    L = 4 * np.pi * d_cm**2 * flux
    
def flux(l, d):
    d_cm = 3.08567758128e21 * float(d) 
#     L = 4 * np.pi * d_cm**2 * flux
    flux = l / (4*np.pi*d_cm**2)
    return flux

print(flux(6.00E+31, 4.8))
print(flux(1.80E+36, 6.3))

2.176495738821969e-14
3.7903599261117297e-10


In [131]:
"{:.3f}".format(0.0038)

'0.004'